# PyCity Schools Analysis

*- Your analysis here
*  
---

In [24]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import numpy.ma.mrecords as mrecords

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [25]:
# Calculate the total number of unique schools
school_count = len(school_data_complete["school_name"].unique())
school_count


15

In [26]:
# Calculate the total number of students
# Note: All students are unique even if they have homonyms in another schools, 
# so the count must consider the repetitions of exact name
student_count=(school_data_complete["student_name"].value_counts()).sum()
student_count

39170

In [27]:
# Calculate the total budget. 
# Budget must fit in each column of school, and the budget could be repeated in more than 1 school
per_school_budget=(school_data_complete['school_name'].groupby(school_data_complete['budget']))
per_school_budget
total_budget=per_school_budget.sum()
total_budget

budget
248087     Holden High SchoolHolden High SchoolHolden Hig...
585858     Pena High SchoolPena High SchoolPena High Scho...
917500     Griffin High SchoolGriffin High SchoolGriffin ...
1043130    Thomas High SchoolThomas High SchoolThomas Hig...
1049400    Wright High SchoolWright High SchoolWright Hig...
1056600    Shelton High SchoolShelton High SchoolShelton ...
1081356    Cabrera High SchoolCabrera High SchoolCabrera ...
1319574    Wilson High SchoolWilson High SchoolWilson Hig...
1763916    Ford High SchoolFord High SchoolFord High Scho...
1884411    Figueroa High SchoolFigueroa High SchoolFiguer...
1910635    Huang High SchoolHuang High SchoolHuang High S...
2547363    Rodriguez High SchoolRodriguez High SchoolRodr...
3022020    Hernandez High SchoolHernandez High SchoolHern...
3094650    Johnson High SchoolJohnson High SchoolJohnson ...
3124928    Bailey High SchoolBailey High SchoolBailey Hig...
Name: school_name, dtype: object

In [28]:
# Calculate the average (mean) math score
average_math_score = school_data_complete["math_score"].mean()
average_math_score


78.98537145774827

In [29]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete["reading_score"].mean()
average_reading_score

81.87784018381414

In [30]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_count
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [31]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"]>=70)].count()["student_name"]
passing_reading_count
passing_reading_percentage = passing_reading_count/float(student_count)*100
passing_reading_percentage

85.80546336482001

In [32]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [33]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({"Total Schools": [school_count],
                                 "Total Students": [student_count],
                                 "Total Budget": [total_budget], 
                                 "Average Math Score": [average_math_score], 
                                 "Average Reading Score": [average_reading_score], 
                                 "% Passing Math": [passing_math_percentage], 
                                 "% Passing Reading": [passing_reading_percentage], 
                                 "% Passing Both": [overall_passing_rate]})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("${:,.2f}".format)
district_summary["Average Reading Score"]=district_summary["Average Reading Score"].map("${:,.2f}".format)
district_summary["% Passing Math"]=district_summary["% Passing Math"].map("${:,.2f}".format)
district_summary["% Passing Reading"]=district_summary["% Passing Reading"].map("${:,.2f}".format)
district_summary["% Passing Both"]=district_summary["% Passing Both"].map("${:,.2f}".format)

# Collect a list of all columns within the DataFrame
district_summary.columns

# Display the DataFrame
district_summary

TypeError: unsupported format string passed to Series.__format__

## School Summary

In [34]:
# Use the code provided to select all of the school types
school_types = school_data_complete["type"].value_counts()
school_types


District    26976
Charter     12194
Name: type, dtype: int64

In [35]:
# Calculate the total student count per school
per_school_counts=school_data_complete.groupby('school_name').count()['student_name']
per_school_counts


school_name
Bailey High School       4976
Cabrera High School      1858
Figueroa High School     2949
Ford High School         2739
Griffin High School      1468
Hernandez High School    4635
Holden High School        427
Huang High School        2917
Johnson High School      4761
Pena High School          962
Rodriguez High School    3999
Shelton High School      1761
Thomas High School       1635
Wilson High School       2283
Wright High School       1800
Name: student_name, dtype: int64

In [36]:
# Calculate the total school budget and per capita spending per school
df=pd.DataFrame(per_school_counts)
df=df.rename(columns={"student_name":"Total students"})
df["Budget per school"]=per_school_budget
df

,Total students,Budget per school
school_name,,
Bailey High School,4976,"(248087, [Holden High School, Holden High Scho..."
Cabrera High School,1858,"(585858, [Pena High School, Pena High School, ..."
Figueroa High School,2949,"(917500, [Griffin High School, Griffin High Sc..."
Ford High School,2739,"(1043130, [Thomas High School, Thomas High Sch..."
Griffin High School,1468,"(1049400, [Wright High School, Wright High Sch..."
Hernandez High School,4635,"(1056600, [Shelton High School, Shelton High S..."
Holden High School,427,"(1081356, [Cabrera High School, Cabrera High S..."
Huang High School,2917,"(1319574, [Wilson High School, Wilson High Sch..."
Johnson High School,4761,"(1763916, [Ford High School, Ford High School,..."


In [37]:
# Calculate the average test scores per school
per_school_math = 
per_school_reading = 

SyntaxError: invalid syntax (4034852390.py, line 2)

In [ ]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math = 
school_students_passing_math = 

In [ ]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = 
school_students_passing_reading = 

In [ ]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()

In [ ]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100

In [ ]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
'''type_summary = pd.DataFrame({"Total Schools": [school_count],
                                 "Total Students": [student_count],
                                 "Total Budget": [budget], 
                                 "Average Math Score": [average_math_score], 
                                 "Average Reading Score": [average_reading_score], 
                                 "% Passing Math": [passing_math_percentage], 
                                 "% Passing Reading": [passing_reading_percentage], 
                                 "% Passing Both": [overall_passing_rate]})'''


per_school_summary = 

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

## Highest-Performing Schools (by % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = 
top_schools.head(5)

## Bottom Performing Schools (By % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = 
bottom_schools.head(5)

## Math Scores by Grade

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = 
tenth_grader_math_scores = 
eleventh_grader_math_scores = 
twelfth_grader_math_scores = 

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = 

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

## Reading Score by Grade 

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = 
tenth_grader_reading_scores = 
eleventh_grader_reading_scores = 
twelfth_grader_reading_scores = 

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = 

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

## Scores by School Spending

In [ ]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [ ]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [ ]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = 
school_spending_df

In [ ]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [ ]:
# Assemble into DataFrame
spending_summary = 

# Display results
spending_summary

## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = 
per_school_summary

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = 

# Display results
size_summary

## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`
'''type_summary = pd.DataFrame({"Total Schools": [school_count],
                                 "Total Students": [student_count],
                                 "Total Budget": [budget], 
                                 "Average Math Score": [average_math_score], 
                                 "Average Reading Score": [average_reading_score], 
                                 "% Passing Math": [passing_math_percentage], 
                                 "% Passing Reading": [passing_reading_percentage], 
                                 "% Passing Both": [overall_passing_rate]})'''

type_summary=

# Display results
type_summary

In [ ]:
# Push the remade DataFrame to a new CSV file
'''renamed_df.to_csv("Output/books_clean.csv",
                  encoding="utf-8", index=False, header=True)'''